# Exploratory data analysis (EDA) of car data

## Libraries and settings

In [13]:
# Libraries
import os
import re
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime, timedelta

# seaborn graphics settings
sns.set_theme(color_codes=True)

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Show current working directory
print(os.getcwd())

u:\Lektionen\GitHub_Repositories\python_exploratory_data_analysis


## Importing and preparing the car data

### Importing the car data

In [14]:
# Read and select variables
df_orig = pd.read_csv("autoscout24_data.csv", sep=";")[["Offer_Id",
                                                        "Type",
                                                        "Price",
                                                        "HP",
                                                        "Mileage",
                                                        "Fuel_Type",
                                                        "Transmission",
                                                        "Dealer_PLZ",
                                                        "Dealer_City",
                                                        "Dealer_Street_House_Nr",
                                                        "Init_Regist_MY",
                                                        "Init_Regist_Month",
                                                        "Init_Regist_Year",
                                                        "Init_Regist_Dt"]]

# Remove duplicates
df_orig = df_orig.drop_duplicates()
df_orig.head(5)

# Remove missing values
df = df_orig.dropna()
df.head(5)

,Offer_Id,Type,Price,HP,Mileage,Fuel_Type,Transmission,Dealer_PLZ,Dealer_City,Dealer_Street_House_Nr,Init_Regist_MY,Init_Regist_Month,Init_Regist_Year,Init_Regist_Dt
0,7324420,AUDI A5 Sportback 3.0 TDI quattro S-tronic (Li...,CHF 22'500.–,245 PS,75'000 km,Diesel,Automatisiertes Schaltgetriebe,8488,Turbenthal,Mettlenstrasse 3,10.2014,10.0,2014.0,2014-10
1,7512768,MERCEDES-BENZ SLK 200 7G-Tronic (Cabriolet),CHF 23'749.–,184 PS,46'655 km,Benzin,Automat sequentiell,3186,Düdingen,Brugerastrasse 60,6.2013,6.0,2013.0,2013-06
2,7512034,MERCEDES-BENZ C 350 Avantgarde 4Matic 7G-Troni...,CHF 18'500.–,306 PS,138'955 km,Benzin,Automat sequentiell,1262,Eysins,1262 Eysins,6.2011,6.0,2011.0,2011-06
3,7512728,MERCEDES-BENZ A 45 AMG 4Matic Speedshift 7G-DC...,CHF 36'000.–,360 PS,43'000 km,Benzin,Automatisiertes Schaltgetriebe,4314,Zeiningen,Am Stutz 21,8.2015,8.0,2015.0,2015-08
4,7490242,AUDI A5 Sportback 2.0 TFSI Sport quattro S-tro...,CHF 48'500.–,252 PS,43'300 km,Benzin,Automatisiertes Schaltgetriebe,3250,Lyss,3250 Lyss,9.2018,9.0,2018.0,2018-09


### Preparing the car data

#### Extract make from car type

In [15]:
# Extract the first word from each string
df['Make'] = df['Type'].str.split().str[0]

# Explanation:
# .split() splits a string, e.g.
# txt = 'AUDI A5 Sportback 3.0 TDI quattro S-tronic'
# txt.split()
# ['AUDI', 'A5', 'Sportback', '3.0', 'TDI', 'quattro', 'S-tronic']
# .str[0] returns the 1st item of the list which is 'AUDI'

# Show result
df[['Type', 'Make']]


,Type,Make
0,AUDI A5 Sportback 3.0 TDI quattro S-tronic (Li...,AUDI
1,MERCEDES-BENZ SLK 200 7G-Tronic (Cabriolet),MERCEDES-BENZ
2,MERCEDES-BENZ C 350 Avantgarde 4Matic 7G-Troni...,MERCEDES-BENZ
3,MERCEDES-BENZ A 45 AMG 4Matic Speedshift 7G-DC...,MERCEDES-BENZ
4,AUDI A5 Sportback 2.0 TFSI Sport quattro S-tro...,AUDI
...,...,...
3975,FERRARI F360 Spider (Cabriolet),FERRARI
3976,MERCEDES-BENZ SL 500 560 SL (Cabriolet),MERCEDES-BENZ
3977,MERCEDES-BENZ CL 63 AMG 7G-Tronic (Coupé),MERCEDES-BENZ
3978,BMW 328i Cabrio (Cabriolet),BMW


#### Extract numerical values from 'Mileage', 'HP' and 'Price'

In [16]:
# Extract values using the lambda function in combination with a regular expression
df['Mileage_num'] = df['Mileage'].apply(lambda x: int(re.sub('[^\d]', '', x)))
df['HP_num'] = df['HP'].apply(lambda x: int(re.sub('[^\d]', '', x)))
df['Price_num'] = df['Price'].apply(lambda x: int(re.sub('[^\d]', '', x)))

# Explanations:
# The lambda function iterates over, e.g. df['Mileage'] in which x is a single string 
# The regular expression '[^\d]' identifies all non numerical characters in x
# The .sub() method replaces the non numerical characters with ''
# The int() converts the resulting string to an integer

# Show result
df[['Mileage', 'Mileage_num', 'HP', 'HP_num', 'Price', 'Price_num']]

,Mileage,Mileage_num,HP,HP_num,Price,Price_num
0,75'000 km,75000,245 PS,245,CHF 22'500.–,22500
1,46'655 km,46655,184 PS,184,CHF 23'749.–,23749
2,138'955 km,138955,306 PS,306,CHF 18'500.–,18500
3,43'000 km,43000,360 PS,360,CHF 36'000.–,36000
4,43'300 km,43300,252 PS,252,CHF 48'500.–,48500
...,...,...,...,...,...,...
3975,78'000 km,78000,400 PS,400,CHF 58'400.–,58400
3976,315'000 km,315000,232 PS,232,CHF 15'900.–,15900
3977,85'000 km,85000,525 PS,525,CHF 42'000.–,42000
3978,188'920 km,188920,193 PS,193,CHF 9'980.–,9980


#### Clean the data by removing missing values and duplicated values

In [17]:
# Remove missing values
df = df.dropna()

# Remove duplicates
df = df.drop_duplicates()

# Show result
print(f'Number of missing values: {df.isna().sum().sum()}')
print('---------------------------------')
print(f'Number of duplicated values: {df.duplicated().sum()}')

Number of missing values: 0
---------------------------------
Number of duplicated values: 0


## Univariate non-graphical exploratory data analysis (EDA)

### Quantiles

### Shape (number of rows and colums)

### Data types

### Summary statistics of numeric variables

### Statistical measures (min, max, std, mean, median, count) for selected variables

### Skewness

### Kurtosis

### Extreme values

### Get a list of categories of categorical variable

## Multivariate non-graphical exploratory data analysis (EDA)

### Cross-tabulation

### Pivot tables

### Correlation matrix

### Covariance matrix

## Univariate graphical exploratory data analysis (EDA)

### Line chart (matplotlib)

### Boxplot (seaborn)

### Histogram  (matplotlib)

### Density plot  (seaborn)

### Quantile-Quantile (QQ-) plot

### Barchart  (matplotlib)

### Piechart (matplotlib)

## Multivariate graphical exploratory data analysis (EDA)

### Scatterplot (matplotlib)

### Scatterplot (matplotlib) with regression line

### Scatterplot-matrix (seaborn)

### Hexagonal binning plot  (matplotlib)

### Correlation heatmap (seaborn)

### Bubble plot (seaborn)

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [18]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
NT
Windows | 10
Datetime: 2024-09-27 17:36:13
Python Version: 3.11.9
-----------------------------------
